<a href="https://colab.research.google.com/github/rebeccalipton/nasa-jam-session/blob/main/reactive_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

from google.colab import files
uploaded = files.upload()

Saving combinedReactiveData.csv to combinedReactiveData (1).csv


In [ ]:
import io
reactive = pd.read_csv(io.BytesIO(uploaded['combinedReactiveData.csv']), sep = ",")
reactive = pd.DataFrame(reactive)

In [ ]:
import numpy as np
# https://stackoverflow.com/questions/19913659/pandas-conditional-creation-of-a-series-dataframe-column
marker_conditions = [
    (reactive['filename'].str.contains("A1AIM")),
    (reactive['filename'].str.contains("A2ASF")),
    (reactive['filename'].str.contains("B1IMAGE")),
    (reactive['filename'].str.contains("B2MESSAGE")),
    (reactive['filename'].str.contains("A1GMINICAM")),
    (reactive['filename'].str.contains("A1TMESSAGE")),
 ]
year_conditions = [
    (reactive['filename'].str.contains("1990")),
    (reactive['filename'].str.contains("2000")),
    (reactive['filename'].str.contains("2010")),
    (reactive['filename'].str.contains("2020")),
    (reactive['filename'].str.contains("2030")),
    (reactive['filename'].str.contains("2040")),
    (reactive['filename'].str.contains("2050")),
    (reactive['filename'].str.contains("2060")),
    (reactive['filename'].str.contains("2070")),
    (reactive['filename'].str.contains("2080")),
    (reactive['filename'].str.contains("2090")),
    (reactive['filename'].str.contains("2100")),
]
gases_conditions = [
    (reactive['filename'].str.contains("CH4")),
    (reactive['filename'].str.contains("CO")),
    (reactive['filename'].str.contains("NOx")),
    (reactive['filename'].str.contains("NMVOC")),
]
marker_choices = ["A1AIM","A2ASF","B1IMAGE","B2MESSAGE","A1GMINICAM","A1TMESSAGE"]
year_choices = ["1990","2000","2010","2020","2030","2040","2050","2060","2070","2080","2090","2100"]
gases_choices = ["CH4","CO","NOx","NMVOC"]
reactive['markers'] = np.select(marker_conditions, marker_choices)
reactive['years'] = np.select(year_conditions, year_choices)
reactive['gases'] = np.select(gases_conditions, gases_choices)


In [ ]:
reactive['filetest'] = reactive.markers + reactive.years + reactive.gases + ".txt"
reactive["identical"] = (reactive.filetest == reactive.filename)

# assures that all of the filenames were split into separate columns correctly since all values returned in identical column are true
reactive.identical.describe() 
reactive.columns

Index(['long', 'lat', 'value', 'filename', 'markers', 'years', 'gases',
       'filetest', 'identical'],
      dtype='object')

In [ ]:
reactiveDF = reactive[['long', 'lat', 'value', 'markers', 'years', 'gases']]
reactiveDF

,long,lat,value,markers,years,gases
0,-170,-20,0.000170,A1TMESSAGE,2070,CH4
1,-167,65,0.000173,A1TMESSAGE,2070,CH4
2,-166,68,0.000053,A1TMESSAGE,2070,CH4
3,-166,65,0.000173,A1TMESSAGE,2070,CH4
4,-166,64,0.000180,A1TMESSAGE,2070,CH4
...,...,...,...,...,...,...
2895979,177,-39,0.022543,B1IMAGE,2090,CH4
2895980,178,63,0.011734,B1IMAGE,2090,CH4
2895981,178,62,0.002186,B1IMAGE,2090,CH4
2895982,178,-18,0.041475,B1IMAGE,2090,CH4


In [132]:
# !pip install -q condacolab
# import condacolab
# condacolab.install()
# !conda install geopandas
# !conda install geopy
# !conda install reverse_geocoder
# !conda install geopy
# !conda install geocoder
# !conda install plotly_express
#!conda install pandas-profiling

In [ ]:
import geopandas
import matplotlib.pyplot as plt

geometry = geopandas.points_from_xy(reactiveDF.long, reactiveDF.lat)
reactiveDF = geopandas.GeoDataFrame(reactiveDF, geometry = geometry)

In [ ]:
# 1st is longitude, 2nd is latitude
#from shapely.geometry import Point
#reactiveDF["address"] = geopandas.tools.reverse_geocode(reactiveDF.geometry)

In [73]:
reactiveDF["geom"] = reactiveDF["lat"].map(str) + ', ' + reactiveDF["long"].map(str)
reactiveDF["geom"] = reactiveDF["geom"].astype(str)

In [127]:
reactiveDF.dtypes

long           int64
lat            int64
value        float64
markers       object
years         object
gases         object
geometry    geometry
geom          object
address       object
dtype: object

In [74]:
# geolocator = Nominatim(user_agent = "get_address")
# location = geolocator.reverse(test.geom)
# location.address

In [126]:
import geopy
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
locator = Nominatim(user_agent = "myGeocoder")


In [128]:
reactiveDF

,long,lat,value,markers,years,gases,geometry,geom,address
0,-170,-20,0.000170,A1TMESSAGE,2070,CH4,POINT (-170.00000 -20.00000),"-20, -170",None
1,-167,65,0.000173,A1TMESSAGE,2070,CH4,POINT (-167.00000 65.00000),"65, -167",None
2,-166,68,0.000053,A1TMESSAGE,2070,CH4,POINT (-166.00000 68.00000),"68, -166",None
3,-166,65,0.000173,A1TMESSAGE,2070,CH4,POINT (-166.00000 65.00000),"65, -166",None
4,-166,64,0.000180,A1TMESSAGE,2070,CH4,POINT (-166.00000 64.00000),"64, -166",None
...,...,...,...,...,...,...,...,...,...
2895979,177,-39,0.022543,B1IMAGE,2090,CH4,POINT (177.00000 -39.00000),"-39, 177",None
2895980,178,63,0.011734,B1IMAGE,2090,CH4,POINT (178.00000 63.00000),"63, 178",None
2895981,178,62,0.002186,B1IMAGE,2090,CH4,POINT (178.00000 62.00000),"62, 178",None
2895982,178,-18,0.041475,B1IMAGE,2090,CH4,POINT (178.00000 -18.00000),"-18, 178",None


In [134]:
# from pandas_profiling import ProfileReport
# profile = ProfileReport(reactiveDF)
# profile